## Mounting drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1. Loading data

In [ ]:
import pickle
import numpy as np
import random

In [ ]:
# open a file, where you stored the pickled data
with open('/content/drive/MyDrive/Colab Notebooks/Data/gnu_data.pkl', 'rb') as file:

    # dump information to that file
    Xd = pickle.load(file,encoding='latin1')


In [ ]:
Xd['X']
Xd['lbl']

X     = []
modul = []
snr   = []

for i in range(len(Xd['X'])):
    x_ = Xd['X'][i]
    X.append(np.reshape(x_,(-1)))
    
    lbl = Xd['lbl'][i]
    modul.append(lbl[0])
    snr.append(int(lbl[1]))

### 1.1. Train/Test Split

In [ ]:
train_test_split = 0.8

random.seed(100)
random.shuffle(X)

random.seed(100)
random.shuffle(modul)

random.seed(100)
random.shuffle(snr)


X_train = np.array(X[:int(train_test_split*len(X))])
X_test  = np.array(X[int(train_test_split*len(X)):])

modul_train = modul[:int(train_test_split*len(X))]
modul_test  = modul[int(train_test_split*len(X)):]

snr_train = snr[:int(train_test_split*len(X))]
snr_test  = snr[int(train_test_split*len(X)):]

### 1.2. Generating One-hot encoded

In [ ]:
mods = list(set(modul_train))
print(len(mods))

def to_onehot(yy):
    yy1 = np.zeros([1, len(mods)])
    yy1[0,mods.index(yy)] = 1
    return yy1

Y_train = np.zeros([len(modul_train),len(mods)])
Y_test = np.zeros([len(modul_test),len(mods)])

for i in range(len(modul_train)):
  Y_train[i,:] = to_onehot(modul_train[i])

for i in range(len(modul_test)):
  Y_test[i,:] = to_onehot(modul_test[i]) 

11


## 2. Train Deep Neural Networks

### 2.1. Train Auto-Encoder (AE)

In [ ]:
import keras

# This is the size of our encoded representations
encoding_dim = 64  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats
data_dim     = (128,2,1,) #X[0].shape[0]

input_data = keras.Input(shape=(data_dim))

# Encoder
x = keras.layers.Conv2D(8, (3, 3), activation="relu", padding="same")(input_data)
x = keras.layers.MaxPooling2D((2, 2), padding="same")(x)
x = keras.layers.Conv2D(32, (3, 3), activation="relu", padding="same")(x)
x = keras.layers.MaxPooling2D((2, 2), padding="same")(x)

# Decoder
x = keras.layers.Conv2DTranspose(32, (3, 3), strides=2, activation="relu", padding="same")(x)
x = keras.layers.Conv2DTranspose(32, (3, 3), strides=2, activation="relu", padding="same")(x)
x = keras.layers.Conv2D(1, (3, 3), activation="sigmoid", padding="same")(x)

# Autoencoder
autoencoder = keras.Model(input_data, x)
autoencoder.compile(optimizer="adam", loss="binary_crossentropy")
autoencoder.summary()

'''# This is our input image
input_data     = keras.Input(shape=(data_dim,))
layer1_encoder = keras.layers.Dense(200, activation='tanh')(input_data)
# layer1_encoder = keras.layers.BatchNormalization()(layer1_encoder)
# layer1_encoder = keras.layers.Dropout(0.3)(layer1_encoder)
layer2_encoder = keras.layers.Dense(200, activation='tanh')(layer1_encoder)
# layer2_encoder = keras.layers.BatchNormalization()(layer2_encoder)
# layer2_encoder = keras.layers.Dropout(0.3)(layer2_encoder)
encoded        = keras.layers.Dense(encoding_dim, activation='tanh',name='encoding')(layer2_encoder)
layer1_decoder = keras.layers.Dense(200, activation='tanh')(encoded)
# layer1_decoder = keras.layers.BatchNormalization()(layer1_decoder)
# layer1_decoder = keras.layers.Dropout(0.3)(layer1_decoder)
layer2_decoder = keras.layers.Dense(200, activation='tanh')(layer1_decoder)
# layer2_decoder = keras.layers.BatchNormalization()(layer2_decoder)
# layer2_decoder = keras.layers.Dropout(0.3)(layer2_decoder)
decoded        = keras.layers.Dense(data_dim, activation='tanh')(layer2_decoder)

# This model maps an input to its reconstruction
autoencoder = keras.Model(input_data, decoded)
autoencoder.compile(loss='mse', optimizer='adam')
autoencoder.summary()'''

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 128, 2, 1)]       0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 128, 2, 8)         80        
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 64, 1, 8)          0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 64, 1, 32)         2336      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 32, 1, 32)         0         
_________________________________________________________________
conv2d_transpose_6 (Conv2DTr (None, 64, 2, 32)         9248      
_________________________________________________________________
conv2d_transpose_7 (Conv2DTr (None, 128, 4, 32)        9248

"# This is our input image\ninput_data     = keras.Input(shape=(data_dim,))\nlayer1_encoder = keras.layers.Dense(200, activation='tanh')(input_data)\n# layer1_encoder = keras.layers.BatchNormalization()(layer1_encoder)\n# layer1_encoder = keras.layers.Dropout(0.3)(layer1_encoder)\nlayer2_encoder = keras.layers.Dense(200, activation='tanh')(layer1_encoder)\n# layer2_encoder = keras.layers.BatchNormalization()(layer2_encoder)\n# layer2_encoder = keras.layers.Dropout(0.3)(layer2_encoder)\nencoded        = keras.layers.Dense(encoding_dim, activation='tanh',name='encoding')(layer2_encoder)\nlayer1_decoder = keras.layers.Dense(200, activation='tanh')(encoded)\n# layer1_decoder = keras.layers.BatchNormalization()(layer1_decoder)\n# layer1_decoder = keras.layers.Dropout(0.3)(layer1_decoder)\nlayer2_decoder = keras.layers.Dense(200, activation='tanh')(layer1_decoder)\n# layer2_decoder = keras.layers.BatchNormalization()(layer2_decoder)\n# layer2_decoder = keras.layers.Dropout(0.3)(layer2_decode

In [ ]:
# Set up some params 
nb_epoch = 100     # number of epochs to train on
batch_size = 32  # training batch size

filepath = '/content/drive/My Drive/amadou/AE.h5'
autoencoder.fit(X_train,
    X_train,
    batch_size=batch_size,
    epochs=nb_epoch,
    verbose=1,
    validation_data=(X_test,X_test),
    callbacks = [
        keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True, mode='auto'),
        keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto')]
    )
# we re-load the best weights once training is finished
autoencoder.load_weights(filepath)

Epoch 1/100


ValueError: ignored

### 2.2. Define Encoder model (trained)

In [ ]:
encoder_model = keras.Model(inputs=autoencoder.input,
                                 outputs=autoencoder.get_layer('encoding').output)

### 2.3. Calculate Encoded data (train & test) using trained Encoder Model

In [ ]:
encoded_train = encoder_model.predict(X_train, verbose=1)
encoded_test  = encoder_model.predict(X_test, verbose=1)


### 2.4. Train MLP Classifier

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Dense(100, activation='relu', name="dense1", input_shape=(encoding_dim,)))
model.add(keras.layers.Dropout(0.1))
model.add(keras.layers.Dense(100, activation='relu', name="dense2"))
model.add(keras.layers.Dropout(0.1))
model.add(keras.layers.Dense( 11, name="output" ))
model.add(keras.layers.Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics='acc')
model.summary()

In [ ]:
filepath = '/content/drive/My Drive/amadou/AE_MLP.h5'

model.fit(encoded_train,
    Y_train,
    batch_size=batch_size,
    epochs=nb_epoch,
    verbose=1,
    validation_data=(encoded_test,Y_test),
    callbacks = [
        keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True, mode='auto'),
        keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto')]
    )
# we re-load the best weights once training is finished
model.load_weights(filepath)

## 3. Evaluating the trained classifier

In [ ]:
snrs= list(set(snr))
snrs.sort()

import matplotlib.pyplot as plt
def plot_confusion_matrix(cm, title='Confusion matrix', cmap=plt.cm.Blues, labels=[]):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(labels))
    plt.xticks(tick_marks, labels, rotation=45)
    plt.yticks(tick_marks, labels)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:

# Plot confusion matrix
acc = {}
for snr_value in snrs:

    # extract classes @ SNR
    test_X_i = X_test[np.where(np.array(snr_test)==snr_value)]
    test_Y_i = Y_test[np.where(np.array(snr_test)==snr_value)]    

    # estimate classes
    test_encoded = encoder_model.predict(test_X_i)
    test_Y_i_hat = model.predict(test_encoded)
    conf = np.zeros([len(mods),len(mods)])
    confnorm = np.zeros([len(mods),len(mods)])
    for i in range(0,test_X_i.shape[0]):
        j = list(test_Y_i[i,:]).index(1)
        k = int(np.argmax(test_Y_i_hat[i,:]))
        conf[j,k] = conf[j,k] + 1
    for i in range(0,len(mods)):
        confnorm[i,:] = conf[i,:] / np.sum(conf[i,:])
    plt.figure()
    plot_confusion_matrix(confnorm, labels=mods, title="AE Confusion Matrix (SNR=%d)"%(int(snr_value)))
    
    cor = np.sum(np.diag(conf))
    ncor = np.sum(conf) - cor
    print ("Overall Accuracy: ", cor / (cor+ncor))
    acc[snr_value] = 1.0*cor/(cor+ncor)

In [ ]:
acc_ = [acc[snr_value] for snr_value in snrs]

# Plot accuracy curve
plt.plot(snrs, acc_)
plt.xlabel("Signal to Noise Ratio")
plt.ylabel("Classification Accuracy")
plt.title("AE Classification Accuracy on RadioML 2016.10 Alpha")